#### 1. CRIM - per capita crime rate by town
#### 2. ZN - proportion of residential land zoned for lots over 25,000 sq.ft.
#### 3. INDUS - proportion of non-retail business acres per town.
#### 4. CHAS - Charles River dummy variable (1 if tract bounds river; 0 otherwise)
#### 5. NOX - nitric oxides concentration (parts per 10 million)
#### 6. RM - average number of rooms per dwelling
#### 7. AGE - proportion of owner-occupied units built prior to 1940
#### 8. DIS - weighted distances to five Boston employment centres
#### 9. RAD - index of accessibility to radial highways
#### 10. TAX - full value property tax rate per $10,000

#### 11. PTRATIO - pupil-teacher ratio by town
#### 12. B - 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
#### 13. LSTAT - % lower status of the population
#### 14. MEDV - Median value of owner-occupied homes in $1000's

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv(r"C:\Users\Vaibhav\OneDrive\Documents\Decode\Project-Housing_Price_Prediction\housing_price_prediction.csv")

In [ ]:
data

In [ ]:
data.shape

In [ ]:
data.describe()

From get-go,  two data coulmns show interesting summeries. They are
* ZN (proportion of residential land zoned for lots over 25,000 sq.ft.)  with 0 for 25th, 50th percentiles.
* Second, CHAS: Charles River dummy variable (1 if tract bounds river; 0 otherwise) with 0 for 25th, 50th and 75th percentiles.
* These summeries are understandable as both variables are conditional as well as categorical variables.


* First assumption would be that these coulms may not be useful in regression task such as predicting MEDV (Median value of owner-occupied homes).
* Another interesing fact on the dataset is the max value of MEDV. From the original data description, it says: Variable #14 seems to be censored at 50.00 (corresponding to a median price of $50,000).
* Based on that, values above 50.00 may not help to predict MEDV. Let's plot the dataset and see interesting trends/stats.

In [ ]:
from scipy import stats

fig, axs = plt.subplots(ncols=7, nrows=2, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in data.items():
    sns.boxplot(y=k, data=data, ax=axs[index])
    index += 1
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

* Columns like CRIM, ZN, RM, B seems to have outliers.
* Let's see the outliers percentage in every column.

In [ ]:
for k, v in data.items():
    q1 = v.quantile(0.25)
    q3 = v.quantile(0.75)
    irq = q3 - q1
    v_col = v[(v <= q1 - 1.5 * irq) | (v >= q3 + 1.5 * irq)]
    perc = np.shape(v_col)[0] * 100.0 / np.shape(data)[0]
    print(f"Column {k} outliers = {round(perc,2)}%")

    
    

### Let's try removing outliers from all the coulmns and then re-plot distribution 

In [ ]:
# col = ["CRIM","ZN","RM","B"]
# fig, axs = plt.subplots(ncols=7, nrows=2, figsize=(20, 10))
# index = 0
# axs = axs.flatten()
# for c in data.columns:
#     if(c in col):
#         percentile25 = data[c].quantile(0.25)
#         percentile75 = data[c].quantile(0.75)
#         iqr = percentile75-percentile25
#         upper_limit = percentile75+(1.5*iqr)
#         lower_limit = percentile25-(1.5*iqr)
#         data = data[data[c]<=upper_limit]
#         data = data[data[c]>=lower_limit]
#         plt.figure()  #for not to overlap other graphs
#         sns.boxplot(y=c, data=data, ax=axs[index])
#         index += 1
#     else:
#         plt.figure()  #for not to overlap other graphs
#         sns.boxplot(y=c, data=data, ax=axs[index])
#         index += 1
# plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)
# plt.show()

### Filling Null values with mean or median, just to reserve data

In [ ]:
# data["INDUS"].fillna(data["INDUS"].mean(), inplace = True)
# data["AGE"].fillna(data["AGE"].mean(), inplace = True)

In [ ]:
# data["CHAS"].fillna(data["CHAS"].median(), inplace = True)
# data["LSTAT"].fillna(data["LSTAT"].median(), inplace = True)

In [ ]:
data.isnull().sum()

Let's see how these features plus MEDV distributions looks like

In [ ]:
fig, axs = plt.subplots(ncols=7, nrows=2, figsize=(20, 10))
index = 0
axs = axs.flatten()
for k,v in data.items():
    sns.distplot(v, ax=axs[index])
    index += 1
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

The histogram also shows that columns CRIM, ZN, B has highly skewed distributions. Also MEDV looks to have a normal distribution (the predictions) and other colums seem to have norma or bimodel ditribution of data except CHAS (which is a discrete variable).

Now let's plot the pairwise  correlation on data.

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(data.corr().abs(),  annot=True)

From correlation matrix, we see TAX and RAD are highly correlated features. The columns LSTAT, INDUS, RM, TAX, NOX, PTRAIO has a correlation score above 0.5 with MEDV which is a good indication of using as predictors. Let's plot these columns against MEDV. 

### Filling Null values with median for reserving data

In [ ]:
x = data[['LSTAT', 'INDUS', 'NOX', 'PTRATIO', 'RM', 'TAX', 'DIS', 'AGE']]
y = data["MEDV"]
for c in x.columns:
    x[c] = x[c].fillna(x[c].median())

### Ploting each column against MEDV

In [ ]:
column_sets = ['LSTAT', 'INDUS', 'NOX', 'PTRATIO', 'RM', 'TAX', 'DIS', 'AGE']
fig, axs = plt.subplots(ncols=4, nrows=2, figsize=(20, 10))
index = 0
axs = axs.flatten()
for i, k in enumerate(column_sets):
    sns.regplot(y=y, x=x[k], ax=axs[i])
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

### As the data points are skewed at one side in most of the columns we will have to do normalization.

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.3,random_state=12)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()

for c in x.columns:
    xtrain[c] = sc.fit_transform(xtrain[[c]])

for c in xtest.columns:
     xtest[c] = sc.fit_transform(xtest[[c]])

In [ ]:
# x2 = x1.drop("MEDV", axis = 1).values
# y2 = x1["MEDV"].values

In [ ]:
# So with these analsis, we may try predict MEDV with 'LSTAT', 'INDUS', 'NOX', 'PTRATIO', 'RM', 'TAX', 'DIS', 'AGE' features.
#Let's try to remove the skewness of the data through log transformation.

In [ ]:
# for c in x.columns:
#     plt.figure()
#     sns.distplot(x[c])
#     plt.show()

In [ ]:
# #y =  np.log1p(y)
# for col in x.columns:
#     if np.abs(x[col].skew()) > 0.3:
#         x[col] = np.log1p(x[col])

# Let's try Linear Regression on dataset first.

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [ ]:
lr.fit(xtrain,ytrain)
ypred = lr.predict(xtest)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
Linear_Regression = r2_score(ytest,ypred)
print("r2 score = ",r2_score(ytest,ypred))
print("MSE = ",mean_squared_error(ytest,ypred))

In [ ]:
print("training score = ",lr.score(xtrain,ytrain))
print("testing score = ",lr.score(xtest,ytest))

# Now lets try Ridge Regression on same data 

In [ ]:
from sklearn.linear_model import Ridge
rg = Ridge()

#### Before fitting model, lets do hyper parameter tuning for getting best parameter

In [ ]:
from sklearn.model_selection import GridSearchCV
para = {"alpha":[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,40,80,100]}
grd = GridSearchCV(rg,para,scoring = "neg_mean_squared_error", cv = 5)
grd.fit(xtrain,ytrain)

In [ ]:
grd.best_params_

In [ ]:
rg = Ridge(alpha = 1)
rg.fit(xtrain,ytrain)
ypred = rg.predict(xtest)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
Ridge_Regression = r2_score(ytest,ypred)
print("r2 score = ",r2_score(ytest,ypred))
print("MSE = ",mean_squared_error(ytest,ypred))

In [ ]:
print("training score = ",rg.score(xtrain,ytrain))
print("testing score = ",rg.score(xtest,ytest))

### So Ridge is giving better score as compare to Linear Regression

### Now lets try for anather regularization technique i.e Lasso

In [ ]:
from sklearn.linear_model import Lasso
ls = Lasso()

### getting best parameter for Lasso as well

In [ ]:
from sklearn.model_selection import GridSearchCV
para = {"alpha":[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,40,80,100]}
grd = GridSearchCV(ls,para,scoring = "neg_mean_squared_error", cv = 5)
grd.fit(xtrain,ytrain)

In [ ]:
grd.best_params_

In [ ]:
ls = Lasso(alpha = 0.01)
ls.fit(xtrain,ytrain)
ypred = ls.predict(xtest)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
Lasso = r2_score(ytest,ypred)
print("r2 score = ",r2_score(ytest,ypred))
print("MSE = ",mean_squared_error(ytest,ypred))

In [ ]:
print("training score = ",ls.score(xtrain,ytrain))
print("testing score = ",ls.score(xtest,ytest))

The Liner Regression with and without L2 regularization does not make significant difference in MSE score.
Let's try some non parametric regression techniques: SVR, DecisionTreeRegressor, KNeighborsRegressor etc.

# SVM

In [ ]:
from sklearn.svm import SVR
svr= SVR()

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# para = {'kernel' : ('linear', 'poly', 'rbf', 'sigmoid'),'C' : [1,5,10],'degree' : [3,8],'coef0' : [0.01,10,0.5],'gamma' : ('auto','scale')}
# rsv = RandomizedSearchCV(svr,para,scoring = "neg_mean_squared_error",n_iter = 5, cv = 5,n_jobs = -1)
# rsv.fit(xtrain,ytrain)

In [ ]:
# rsv.best_params_

In [ ]:
svr= SVR(kernel = 'rbf', gamma = 'scale', degree = 8, coef0 = 0.1, C = 10)
svr.fit(xtrain,ytrain)
ypred = svr.predict(xtest)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
SVR = r2_score(ytest,ypred)
print("r2 score = ",r2_score(ytest,ypred))
print("MSE = ",mean_squared_error(ytest,ypred))

In [ ]:
print("training score = ",ls.score(xtrain,ytrain))
print("testing score = ",ls.score(xtest,ytest))

# Trying Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dc = DecisionTreeRegressor()

In [ ]:
grd = GridSearchCV(dc, cv = 5, param_grid={"max_depth" : [3, 4, 5, 6, 7,8,9]}, scoring='neg_mean_squared_error')
grd.fit(xtrain,ytrain)

In [ ]:
grd.best_params_

In [ ]:
dc = DecisionTreeRegressor(max_depth =  6)
dc.fit(xtrain,ytrain)
ypred = dc.predict(xtest)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
Decision_Tree_Regressor = r2_score(ytest,ypred)
print("r2 score = ",r2_score(ytest,ypred))
print("MSE = ",mean_squared_error(ytest,ypred))

In [ ]:
print("training score = ",ls.score(xtrain,ytrain))
print("testing score = ",ls.score(xtest,ytest))

# KNeighborsRegressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()

In [ ]:
grd = GridSearchCV(knn, cv = 5, param_grid={"n_neighbors" : [2, 3, 4, 5, 6, 7]}, scoring='neg_mean_squared_error')
grd.fit(xtrain,ytrain)

In [ ]:
grd.best_params_

In [ ]:
knn = KNeighborsRegressor(n_neighbors = 7)
knn.fit(xtrain,ytrain)
ypred = knn.predict(xtest)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
KNeighborsRegressor = r2_score(ytest,ypred)
print("r2 score = ",r2_score(ytest,ypred))
print("MSE = ",mean_squared_error(ytest,ypred))

In [ ]:
print("training score = ",ls.score(xtrain,ytrain))
print("testing score = ",ls.score(xtest,ytest))

Compared to three models which are chosen through grid search, SVR performes better. Let's try an ensemble method finally.

# Gradient Boosting Regressor

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor()

In [ ]:
para ={'n_estimators':[100, 200], 'learning_rate': [0.1,0.05,0.02], 'max_depth':[2, 4,6], 'min_samples_leaf':[3,5,9]}
grd = GridSearchCV(gbr, cv = 5, param_grid = para, scoring='neg_mean_squared_error',n_jobs = -1)
grd.fit(xtrain,ytrain)
grd.best_params_

In [ ]:
gbr = GradientBoostingRegressor(alpha=0.9,learning_rate=0.1, max_depth=2, min_samples_leaf=3, min_samples_split=2, n_estimators=200, random_state=30)
gbr.fit(xtrain,ytrain)
ypred = gbr.predict(xtest)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
Gradient_Boosting_Regressor = r2_score(ytest,ypred)
print("r2 score = ",r2_score(ytest,ypred))
print("MSE = ",mean_squared_error(ytest,ypred))

In [ ]:
print("training score = ",ls.score(xtrain,ytrain))
print("testing score = ",ls.score(xtest,ytest))

Let's plot k-fold results to see which model has better distribution of results. Let's have a look at the MSE distribution of these models with k-fold=10

In [ ]:
# plt.figure(figsize=(20, 10))
# scores_map = pd.DataFrame(scores_map)
# sns.boxplot(data=scores_map)

In [ ]:
df = pd.DataFrame()

df["Linear_Regression"] = Linear_Regression
df["Ridge_Regression"] = Ridge_Regression
df["Lasso"] = Lasso
df["SVR"] = SVR
df["Decision_Tree_Regressor"] = Decision_Tree_Regressor
df["KNeighborsRegressor"] = KNeighborsRegressor
df["Gradient_Boosting_Regressor"] = Gradient_Boosting_Regressor

In [ ]:
#df = pd.DataFrame()

new = {"Linear_Regression" : Linear_Regression,
"Ridge_Regression" : Ridge_Regression,
"Lasso" : Lasso,
"SVR" : SVR,
"Decision_Tree_Regressor" : Decision_Tree_Regressor,
"KNeighborsRegressor" : KNeighborsRegressor,
"Gradient_Boosting_Regressor" : Gradient_Boosting_Regressor
      }

In [ ]:
df = df.append(new, ignore_index=True)

In [ ]:
df = df.T

In [ ]:
df = df.reset_index()

In [ ]:
df.columns = ["Model" , "R2_Score"] 

In [ ]:
df[df["R2_Score"]==df["R2_Score"].max()]

# So we come to Conclusion that SVM's SVR is the best model for our Data Set with the highest accuracy . 

### Load Pickled Model

In [ ]:
import pickle as pk

In [ ]:
filename = "trained_model_svr.sav"
pk.dump(svr, open(filename, "wb"))

##### Loading the saved model

In [ ]:
loaded_model = pk.load(open("trained_model_svr.sav", "rb"))
data.head(1)

In [ ]:
# ['LSTAT', 'INDUS', 'NOX', 'PTRATIO', 'RM', 'TAX', 'DIS', 'AGE']

In [ ]:
def hpp(input_data):
    # input_data = (4.98, 2.31, 0.538, 15.3, 6.575, 296, 4.09, 65)
    arr = np.asarray(input_data)
    arr = arr.reshape(1, -1)
    ar = sc.fit_transform(arr)

    prediction = loaded_model.predict(ar)
    #res = ("The Median value of owner-occupied homes in $1000's is ", prediction)
    return(f"The Median value of owner-occupied homes in $1000's is {round(prediction[0],2)}")



hpp([4.98,2.31,0.538,15.3,6.575,296,4.09,65])